In [1]:
!pip freeze | grep nvid

nvidia-cublas-cu12==12.1.3.1
nvidia-cuda-cupti-cu12==12.1.105
nvidia-cuda-nvrtc-cu12==12.1.105
nvidia-cuda-runtime-cu12==12.1.105
nvidia-cudnn-cu12==9.1.0.70
nvidia-cufft-cu12==11.0.2.54
nvidia-curand-cu12==10.3.2.106
nvidia-cusolver-cu12==11.4.5.107
nvidia-cusparse-cu12==12.1.0.106
nvidia-dali-cuda100==0.27.0
nvidia-dali-cuda110==1.39.0
nvidia-dali-cuda120==1.38.0
nvidia-modulus==0.6.0
nvidia-nccl-cu12==2.20.5
nvidia-nvimgcodec-cu11==0.2.0.7
nvidia-nvimgcodec-cu12==0.2.0.7
nvidia-nvjitlink-cu12==12.6.20
nvidia-nvtx-cu12==12.1.105
nvidia-pyindex==1.0.9


In [2]:
!pip freeze | grep modulus

modulus==0.0.10a0
nvidia-modulus==0.6.0


In [3]:
#!pip install requests

In [4]:
import modulus

In [5]:
#!pip install warp-lang

In [6]:
from modulus.datapipes.benchmarks.darcy import Darcy2D
from modulus.metrics.general.mse import mse
from modulus.models.fno.fno import FNO

Warp 1.2.2 initialized:
   CUDA Toolkit 11.8, Driver 12.5
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla V100-SXM2-32GB" (32 GiB, sm_70, mempool enabled)
   Kernel cache:
     /home/gtamkin/.cache/warp/1.2.2


In [8]:
import torch

In [9]:
normaliser = {
    "permeability": (1.25, 0.75),
    "darcy": (4.52e-2, 2.79e-2),
}
dataloader = Darcy2D(
    resolution=256, batch_size=64, nr_permeability_freq=5, normaliser=normaliser
)
model = FNO(
    in_channels=1,
    out_channels=1,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=4,
    num_fno_modes=12,
    padding=5,
).to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=lambda step: 0.85**step
)

# run for 20 iterations
for i in range(20):
    batch = next(iter(dataloader))
    true = batch["darcy"]
    pred = model(batch["permeability"])
    loss = mse(pred, true)
    loss.backward()
    optimizer.step()
    scheduler.step()

    print(f"Iteration:{i}. Loss:{loss.detach().cpu().numpy()}")

Module modulus.datapipes.benchmarks.kernels.initialization dd354b8 load on device 'cuda:0' took 34.49 ms
Module modulus.datapipes.benchmarks.kernels.utils 68c2b86 load on device 'cuda:0' took 30.55 ms
Module modulus.datapipes.benchmarks.kernels.finite_difference 3584cc4 load on device 'cuda:0' took 28.81 ms
Iteration:0. Loss:0.9422825574874878
Iteration:1. Loss:1.0521711111068726
Iteration:2. Loss:0.9511062502861023
Iteration:3. Loss:0.8808196783065796
Iteration:4. Loss:0.7981544137001038
Iteration:5. Loss:0.6704221367835999
Iteration:6. Loss:0.4890911281108856
Iteration:7. Loss:0.32805773615837097
Iteration:8. Loss:0.4032154679298401
Iteration:9. Loss:0.8170997500419617
Iteration:10. Loss:0.862799882888794
Iteration:11. Loss:0.6109097003936768
Iteration:12. Loss:0.3110634386539459
Iteration:13. Loss:0.1834431290626526
Iteration:14. Loss:0.1307375431060791
Iteration:15. Loss:0.1278097927570343
Iteration:16. Loss:0.16044887900352478
Iteration:17. Loss:0.21152685582637787
Iteration:18. L